## Numerical lookup table for Gaussian sigma from Rician median
Mike Tyszka | CBIC | 2025-12-18

In [2]:
import numpy as np
from scipy.stats import mode

In [ ]:
# Create a complex-valued array with normally distributed real and imaginary parts
n = 1000

# Standard deviation of the Gaussian noise
sigma = 1.0

# Number of levels of SNR to simulate
n_snr = 20

median_residual = np.zeros(n_snr)

# For sigma = 1 the offset and SNR are identical
for ic, snr in enumerate(np.linspace(0.0, 20.0, n_snr)):
   
    # Complex Rician noise
    z_noise = np.random.normal(0, sigma, (n, n)) + 1j * np.random.normal(0, sigma, (n, n)) + snr

    # Calculate the magnitude of the complex-valued array
    magnitude = np.abs(z_noise)

    # Calculate the median of the magnitude
    median_residual[ic] = np.median(magnitude-snr)

    print(f"SNR {snr:0.2f} => median residual {median_residual[ic]:0.2f}")

SNR 0.00 => median residual 1.18
SNR 1.05 => median residual 0.46
SNR 2.11 => median residual 0.23
SNR 3.16 => median residual 0.16
SNR 4.21 => median residual 0.12
SNR 5.26 => median residual 0.09
SNR 6.32 => median residual 0.08
SNR 7.37 => median residual 0.07
SNR 8.42 => median residual 0.06
SNR 9.47 => median residual 0.05
SNR 10.53 => median residual 0.05
SNR 11.58 => median residual 0.04
SNR 12.63 => median residual 0.04
SNR 13.68 => median residual 0.04
SNR 14.74 => median residual 0.04
SNR 15.79 => median residual 0.03
SNR 16.84 => median residual 0.03
SNR 17.89 => median residual 0.03
SNR 18.95 => median residual 0.03
SNR 20.00 => median residual 0.02


In [4]:
# Report descriptive statistics
print(f'Complex Gaussian noise parameters: mean=0, std={sigma}, size=({n}, {n})')

mean_theoretical = sigma * np.sqrt(np.pi / 2)
median_theoretical = sigma * np.sqrt(2 * np.log(2))
mad_theoretical = sigma * 0.6745
mode_theoretical = sigma
std_theoretical = sigma * np.sqrt((4 - np.pi) / 2)

mean_numeric = np.mean(magnitude)
median_numeric = np.median(magnitude)
std_numeric = np.std(magnitude)

# Get mode from histogram maximum
mode_index = np.argmax(density)
mode_numeric = (bins[mode_index] + bins[mode_index + 1]) / 2

# Compare theoretical and numeric values
print(f'Mean: {mean_theoretical:.4f} ({mean_numeric:.4f})')
print(f'Median: {median_theoretical:.4f} ({median_numeric:.4f})')
print(f'Mode: {mode_theoretical:.4f} ({mode_numeric:.4f})')
print(f'Std: {std_theoretical:.4f} ({std_numeric:.4f})')

Complex Gaussian noise parameters: mean=0, std=9.0, size=(2000, 2000)
Mean: 11.2798 (11.2808)
Median: 10.5967 (10.5993)
Mode: 9.0000 (8.9167)
Std: 5.8962 (5.8931)
